In [4]:
## import 部分  
##%overwritefile
##%file:../src/head.py
##%noruncode

from math import exp
from queue import Queue
from threading import Thread
from ipykernel.kernelbase import Kernel
from pexpect import replwrap, EOF
from jinja2 import Environment, PackageLoader, select_autoescape,Template
from abc import ABCMeta, abstractmethod
from typing import List, Dict, Tuple, Sequence
from shutil import copyfile,move
from urllib.request import urlopen
import base64
import urllib.request
import urllib.parse
import platform
import pexpect
import signal
import typing 
import typing as t
import re
import signal
import subprocess
import tempfile
import os
import stat
import sys
import traceback
import os.path as path
import codecs
import time
import importlib
import importlib.util
import inspect

from plugins.ISpecialID import IStag,IDtag,IBtag,ITag,ICodePreproc
from plugins._filter2_magics import Magics

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/head.py created successfully


In [5]:
## 全局函数和变量
##%overwritefile
##%file:../src/common.py
##%noruncode


##%test_begin

##%test_end

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/common.py created successfully


In [6]:
## 核心内变量与属性  
##%overwritefile
##%file:../src/comm_attribute.py
##%noruncode
    pausestr='''
get_char()
{
SAVEDSTTY=`stty -g`
stty -echo
stty cbreak
dd if=/dev/tty bs=1 count=1 2> /dev/null
stty -raw
stty echo
stty $SAVEDSTTY
}
echo ""
echo "Press any key to start...or Press Ctrl+c to cancel"
char=`get_char`
echo "OK"
'''
    silent=None
    jinja2_env = Environment()
    g_rtsps={}
    g_chkreplexit=True

    def get_retinfo(self, rettype:int=0):
        retinfo={'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
        return retinfo

    def chkjoptions(self,magics,jarfile,targetdir):
        if len(self.addmagicsSkey(magics,'joptions'))>-1:
            index=-1
            try:
                index=self.addmagicsSkey(magics,'joptions').index('-cp')
            except Exception as e:
                pass
            if(index<0):
                magics['_st']['joptions']+=['-cp']
                magics['_st']['joptions']+=[':']
                index=index+1
            cpstr=magics['_st']['joptions'][index+1]
            cpstr=cpstr+":"+jarfile+":"+targetdir
            if cpstr.strip().startswith(':'):
                cpstr=cpstr[1:] 
            # self._log(cpstr)
            magics['_st']['joptions'][index+1]=cpstr

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/comm_attribute.py created successfully


In [7]:
## 解析环境变量参数字符串函数
##%overwritefile
##%file:../src/resolving_enveqval.py
##%noruncode
    def resolving_enveqval(self, envstr):
        if envstr is None or len(envstr.strip())<1:
            return os.environ
        # env_dict={}
        argsstr=self.replacemany(self.replacemany(self.replacemany(envstr.strip(),('  '),' '),('= '),'='),' =','=')
        pattern = re.compile(r'([^\s*]*)="(.*?)"|([^\s*]*)=(\'.*?\')|([^\s*]*)=(.[^\s]*)')
        for argument in pattern.findall(argsstr):
            li=list(argument)
            li= [i for i in li if i != '']
            # env_dict[str(li[0])]=li[1]
            os.environ.setdefault(str(li[0]),li[1])
        # envstr=str(str(envstr.split("|")).split("=")).replace(" ","").replace("\'","").replace("\"","").replace("[","").replace("]","").replace("\\","")
        # env_list=envstr.split(",")
        # for i in range(0,len(env_list),2):
        #     os.environ.setdefault(env_list[i],env_list[i+1])
        return os.environ


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/resolving_enveqval.py created successfully


In [8]:
## 解析参数字符串函数
##%overwritefile
##%file:../src/resolving_eqval2dict.py
##%noruncode
    def resolving_eqval2dict(self,argsstr):
        if not argsstr or len(argsstr.strip())<1:
            return None
        env_dict={}
        argsstr=self.replacemany(self.replacemany(self.replacemany(argsstr.strip(),('  '),' '),('= '),'='),' =','=')
        pattern = re.compile(r'([^\s*]*)="(.*?)"|([^\s*]*)=(\'.*?\')|([^\s*]*)=(.[^\s]*)')
        for argument in pattern.findall(argsstr):
            li=list(argument)
            li= [i for i in li if i != '']
            env_dict[str(li[0])]=li[1]
        return env_dict

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/resolving_eqval2dict.py created successfully


In [9]:
## 清除注释函数
##%overwritefile
##%file:../src/cleannotes.py
##%noruncode
    def _is_test_begin(self,line):
        if line==None or line=='':return ''
        return line.strip().startswith('##test_begin') or line.strip().startswith('//test_begin')
    def _is_test_end(self,line):
        if line==None or line=='':return ''
        return line.strip().startswith('##test_end') or line.strip().startswith('//test_end')
    def _is_dqm_begin(self,line):
        if line==None or line=='':return ''
        line=self.replacemany(line.strip(),(' '),'')
        if '=\"\"\"' in line: 
            self.isdstr=True
            return False
        if '\"\"\"' in line: 
            if self.isdstr:return False
            self.isdstr=False
            return True
        return line.lstrip().startswith('\"\"\"')
    def _is_dqm_end(self,line):
        if line==None or line=='':return ''
        if self.isdqm:
            return line.rstrip().endswith('\"\"\"')
        return False
    def _is_sqm_begin(self,line):
        if line==None or line=='':return ''
        line=self.replacemany(line.strip(),(' '),'')
        if '=\'\'\'' in line: 
            self.issstr=True
            return False
        if '\'\'\'' in line: 
            if self.issstr:return False
            self.issstr=False
            return True
        return line.lstrip().startswith('\'\'\'')
    def _is_sqm_end(self,line):
        if line==None or line=='':return ''
        if self.issqm:
            return line.rstrip().endswith('\'\'\'')
        return False
    
    ## 清除C里的多行注释
    def cleanCdqm(self,code):
        return re.sub(r"/\*.*?\*/", "", code, flags=re.M|re.S)
    ## 清除单行注释
    def cleanCnotes(self,code):
        return re.sub(r"//.*", "", code)
    def cleannotes(self,line):
        ##tmpCode = re.sub(r"//.*", "", line)
        ##tmpCode = re.sub(r"/\*.*?\*/", "", tmpCode, flags=re.M|re.S)
        return '' if (not self._is_specialID(line)) and (line.lstrip().startswith('## ') or line.lstrip().startswith('// ')) else line
    isdqm=False##清除双引号多行注释
    def cleandqmA(self,code):
        return re.sub(r"\"\"\".*?\"\"\"", "", code, flags=re.M|re.S)
    def cleandqm(self,line):
        ##tmpCode = re.sub(r"\"\"\".*?\"\"\"", "", line, flags=re.M|re.S)
        if not self.isdqm:
            istb=self._is_dqm_begin(line)
            if istb: 
                self.isdqm=True
                if len(line.strip())>5:
                    iste=self._is_dqm_end(line)
                    if iste:self.isdqm=False
                return ''
        iste=self._is_dqm_end(line)
        if iste:
            self.isdqm=False
            return ''
        line= "" if self.isdqm else line
        return line
    issqm=False
    ## 清除单引号多行注释
    def cleansqmA(self,code):
        return re.sub(r"\'\'\'.*?\'\'\'", "", code, flags=re.M|re.S)
    def cleansqm(self,line):
        ## tmpCode = re.sub(r"\'\'\'.*?\'\'\'", "", line, flags=re.M|re.S)
        if not self.issqm:
            istb=self._is_sqm_begin(line)
            if istb: 
                self.issqm=True
                if len(line.strip())>5:
                    iste=self._is_sqm_end(line)
                    if iste:self.issqm=False
                return ''
        iste=self._is_sqm_end(line)
        if iste:
            self.issqm=False
            return ''
        line= "" if self.issqm else line
        return line
    istestcode=False
    ## 清除测试行里的代码 ##test_begin  ##test_end
    def cleantestcodeA(self,code):
        code=re.sub(r"\/\/test_begin.*?\/\/test_end", "", code, flags=re.M|re.S)
        return re.sub(r"\#\#test_begin.*?\#\#test_end", "", code, flags=re.M|re.S)
    def cleantestcode(self,line):
        if not self.istestcode:
            istb=self._is_test_begin(line)
            if istb: 
                self.istestcode=True
                if len(line.strip())>5:
                    iste=self._is_test_end(line)
                    if iste:self.istestcode=False
                return ''
        iste=self._is_test_end(line)
        if iste:
            self.istestcode=False
            return ''
        line= "" if self.istestcode else line
        return line
    

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/cleannotes.py created successfully


In [10]:
## 判断是否为特殊标识符号
##%overwritefile
##%file:../src/_is_specialID.py
##%noruncode
def _is_specialID(self,line):
    if line.strip().startswith('##%') or line.strip().startswith('//%'):
        return True
    return False


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_is_specialID.py created successfully


In [11]:
## 内核公共代码部分2
##%overwritefile
##%file:../src/kernel_common2.py
##%noruncode
    def get_outencode(self,magics):
        encodestr=self.get_magicsSvalue(magics,"outencode")
        if len(encodestr)<1:
            encodestr='UTF-8'
        return encodestr
    def get_magicsSvalue(self,magics:Dict,key:str):
        return self.addmagicsSkey(magics,key)
    def get_magicsBvalue(self,magics:Dict,key:str):
        return self.addmagicsBkey(magics,key)
    def get_magicsbykey(self,magics:Dict,key:str):
        return self.addkey2dict(magics,key)
    
    def addmagicsSLkey(self,magics:Dict,key:str,value=None,func=None):
        return self.addmagicskey2(magics=magics,key=key,type='_sline',func=func,value=value)
    def addmagicsSkey(self,magics:Dict,key:str,func=None):
        return self.addmagicskey2(magics=magics,key=key,type='_st',func=func)
    def addmagicsBkey(self,magics:Dict,key:str,value=None,func=None):
        return self.addmagicskey2(magics=magics,key=key,type='_bt',func=func,value=value)
    
    def addmagicskey2(self,magics:Dict,key:str,type:str,func=None,value=None):
        if not magics[type].__contains__(key):
            ##添加 key
            d={key:[]}
            if value!=None:
                d={key:value}
            magics[type].update(d)
        if not magics[type+'f'].__contains__(key):
            ##添加 key相关回调函数
            d={key:[]}
            magics[type+'f'].update(d)
        if func!=None:
            magics[type+'f'][key]+=[func]
        return magics[type][key]
    def addkey2dict(self,magics:Dict,key:str,type:str=None):
        if not magics.__contains__(key):
            d={key:[]}
            if type!=None and type=="dict":
                d={key:{}}
            magics.update(d)
        return magics[key]

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/kernel_common2.py created successfully


In [12]:
## 内核公共代码部分
##%overwritefile
##%file:../src/kernel_common.py
##%noruncode
    usleep = lambda x: time.sleep(x/1000000.0)


    ## 全部替换our_str字符串里的to_be_replaced为replace_with
    def replacemany(self,our_str, to_be_replaced:str, replace_with:str):
        while (to_be_replaced in our_str):
            our_str = our_str.replace(to_be_replaced, replace_with)
        return our_str
    ## 处理acb=xxx 这样的参数字符串到字典{}里
    def _filter_dict(self,argsstr):
        if not argsstr or len(argsstr.strip())<1:
            return None
        env_dict={}
        argsstr=self.replacemany(self.replacemany(self.replacemany(argsstr.strip(),('  '),' '),('= '),'='),' =','=')
        pattern = re.compile(r'([^\s*]*)="(.*?)"|([^\s*]*)=(\'.*?\')|([^\s*]*)=(.[^\s]*)')
        for argument in pattern.findall(argsstr):
            li=list(argument)
            li= [i for i in li if i != '']
            env_dict[str(li[0])]=li[1]
        return env_dict
    ## 文件处理器
    def _fileshander(self,files:List,srcfilename,magics)->str:
        index=-1
        fristfile=srcfilename
        try:
            for newsrcfilename in files:
                index=index+1
                newsrcfilename = os.path.join(os.path.abspath(''),newsrcfilename)
                if os.path.exists(newsrcfilename):
                    if magics!=None and len(self.addkey2dict(magics,'overwritefile'))<1:
                        newsrcfilename +=".new.py"
                if not os.path.exists(os.path.dirname(newsrcfilename)) :
                    os.makedirs(os.path.dirname(newsrcfilename))
                if index==0:
                    os.rename(srcfilename,newsrcfilename)
                    fristfile=newsrcfilename
                    files[0]=newsrcfilename
                else:
                    self._write_to_stdout("copy to :"+newsrcfilename+"\n")
                    copyfile(fristfile,newsrcfilename)
        except Exception as e:
                self._log(str(e),2)
        return files[0]

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/kernel_common.py created successfully


In [13]:
## chk_replexit
##%overwritefile
##%file:../src/chk_replexit.py
##%noruncode
    def chk_replexit(self,grtsps): 
        while self.g_chkreplexit:
            try:
                if len(grtsps)>0: 
                    for key in grtsps:
                        if grtsps[key].child.terminated:
                            pass
                            del grtsps[key]
                        ## else:
                        ##     grtsps[key].write_contents()
            finally:
                pass
        if len(grtsps)>0: 
            for key in grtsps:
                if grtsps[key].child.terminated:
                    pass
                    del grtsps[key]
                else:
                    grtsps[key].child.terminate(force=True)
                    del grtsps[key]

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/chk_replexit.py created successfully


In [14]:
## 清除执行过程中生存的临时文件
##%overwritefile
##%file:../src/cleanup_files.py
##%noruncode
    def cleanup_files(self):
        """Remove all the temporary files created by the kernel"""
        # keep the list of files create in case there is an exception
        # before they can be deleted as usual
        if self.files != None and len(self.files) > 0:
            for file in self.files:
                if(os.path.exists(file)):
                    os.remove(file)



[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/cleanup_files.py created successfully


In [15]:
## 产生一个新的临时文件
##%overwritefile
##%file:../src/new_temp_file.py
##%noruncode
    def new_temp_file(self, **kwargs):
        """Create a new temp file to be deleted when the kernel shuts down"""
        # We don't want the file to be deleted when closed, but only when the kernel stops
        kwargs['delete'] = False
        kwargs['mode'] = 'w'
        file = tempfile.NamedTemporaryFile(**kwargs)
        self.files.append(file.name)
        return file
    def create_codetemp_file(self,magics,code,suffix):
        encodestr='UTF-8'
        if magics!=None:
            encodestr=self.get_magicsSvalue(magics,"fileencode")
        if len(encodestr)<1:
            encodestr='UTF-8'
        if (suffix.strip().lower().endswith(".bat") or
            (suffix.strip().lower().endswith(".ps1") and self.sys=="Windows")):
            encodestr="GBK"

        source_file=self.new_temp_file(suffix=suffix,dir=os.path.abspath(''),encoding=encodestr)
        magics['codefilename']=source_file.name
        with  source_file:
            source_file.write(code)
            source_file.flush()
        return source_file

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/new_temp_file.py created successfully


In [16]:
## 按 mimetype 格式输出显示内容
##%overwritefile
##%file:../src/_write_display_data.py
##%noruncode
    def _write_display_data(self,mimetype='text/html',contents=""):
        try:
            if mimetype.startswith('image'):
                metadata ={mimetype:{}}
                # contents=contents
                # self._logln(base64.encodebytes(contents))
                # contents=base64.encodebytes(contents)
                # contents=urllib.parse.quote(base64.b64encode(contents))
                header="<div><img alt=\"Output\" src=\"data:"+mimetype+";base64,"
                end="\"></div>"
                contents=header+base64.encodebytes(contents).decode( errors='ignore')+end
                mimetype='text/html'
                metadata = {mimetype:{}}
                    ## 'text/html' : {
                    ## 'width': 640,
                    ## 'height': 480
                    ## }
                    ## }
        except Exception as e:
            self._logln("_write_display_data err "+str(e),3)
            return
        self.send_response(self.iopub_socket, 'display_data', {'data': {mimetype:contents}, 'metadata': {mimetype:{}}})


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_write_display_data.py created successfully


In [17]:
## 输出结果信息
##%overwritefile
##%file:../src/_write_to_stdout.py
##%noruncode
    def _write_to_stdout(self,contents,magics=None):
        if magics !=None and len(magics['_st']['outputtype'])>0:
            self._write_display_data(mimetype=magics['_st']['outputtype'],contents=contents)
        else:
            self.send_response(self.iopub_socket, 'stream', {'name': 'stdout', 'text': contents})


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_write_to_stdout.py created successfully


In [18]:
## 输出错误信息
##%overwritefile
##%file:../src/_write_to_stderr.py
##%noruncode
    def _write_to_stderr(self, contents):
        self.send_response(self.iopub_socket, 'stream', {'name': 'stderr', 'text': contents})


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_write_to_stderr.py created successfully


In [19]:
## 产生一个文本输入框
##%overwritefile
##%file:../src/_read_from_stdin.py
##%noruncode
    def _read_from_stdin(self):
        return self.raw_input()


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_read_from_stdin.py created successfully


In [20]:
## 读取代码文件，include使用
##%overwritefile
##%file:../src/readcodefile.py
##%noruncode
    def readcodefile(self,filename,spacecount=0):
        filecode=''
        codelist1=None
        if not os.path.exists(filename):
            return ''
        with open(os.path.join(os.path.abspath(''),filename), 'r',encoding="UTF-8") as codef1:
            codelist1 = codef1.readlines()
        if len(codelist1)>0:
            for t in codelist1:
                filecode+=' '*spacecount + t
        return filecode
    def loadurl(self,url):
        content=''
        try:
            request=urllib.request.Request(url)
            myURL = urlopen(request)
            # content= myURL.read()
            lines = myURL.readlines()
            for line in lines:
                print(line)
                content+=line.decode()+"\n"
        except Exception as e:
            self._logln("loadurl error! "+str(e),3)
        return content


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/readcodefile.py created successfully


In [21]:
## 输出信息
##%overwritefile
##%file:../src/process_output.py
##%noruncode
    def process_output(self, output,magics=None):
        if not self.silent:
            if magics !=None and len(magics['_st']['outputtype'])>0:
                self._write_display_data(mimetype=magics['_st']['outputtype'],contents=output)
                return
            # Send standard output
            stream_content = {'name': 'stdout', 'text': output}
            self.send_response(self.iopub_socket, 'stream', stream_content)


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/process_output.py created successfully


In [22]:
## 输出日志信息
##%overwritefile
##%file:../src/log_output.py
##%noruncode
    ## _loglevel='1'
    def _log(self, output,level=1,outputtype='text/plain'):
        if self._loglevel=='0': return
        streamname='stdout'
        if not self.silent:
            prestr=self.kernelinfo+' Info:'
            if level==2:
                prestr=self.kernelinfo+' Warning:'
                streamname='stderr'
            elif level==3:
                prestr=self.kernelinfo+' Error:'
                streamname='stderr'
            else:
                prestr=self.kernelinfo+' Info:'
                streamname='stdout'
            # if len(outputtype)>0 and (level!=2 or level!=3):
                # self._write_display_data(mimetype=outputtype,contents=prestr+output)
                # return
            # Send standard output
            stream_content = {'name': streamname, 'text': prestr+output}
            self.send_response(self.iopub_socket, 'stream', stream_content)
    def _logln(self, output,level=1,outputtype='text/plain'):
        self._log(output+"\n",level=1,outputtype='text/plain')

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/log_output.py created successfully


In [23]:
## send_cmd
##%overwritefile
##%file:../src/send_cmd.py
##%noruncode
    def send_cmd(self,pid,cmd):
        try:
            # self._write_to_stdout("send cmd PID:"+pid+"\n cmd:"+cmd)
            # if self.g_rtsps.has_key(pid):
                # self._write_to_stderr("[MyPythonkernel] Info:exist! "+pid+"\n")
            # self.g_rtsps[pid].stdin.write(cmd.encode())
            self.g_rtsps[pid]._write_to_stdout(cmd)
        except Exception as e:
            self._log("Executable send_cmd error! "+str(e)+"\n")
    
        return


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/send_cmd.py created successfully


In [4]:
## create_jupyter_subprocess
##%overwritefile
##%file:../src/create_jupyter_subprocess.py
##%noruncode
    def create_jupyter_subprocess(self, cmd,cwd=None,shell=False,env=None,magics=None,outencode=None):
        try:
            if env==None or len(env)<1:
                env=os.environ
            if magics!=None and magics['status']=='' and len(self.addmagicsBkey(magics,'runinterm'))>0:
                self.inittermcmd(magics)
                if len(magics['_st']['term'])<1:
                    self._logln("no term！",2)
                execfile=''
                for x in cmd:
                    execfile+=x+" "
                cmdshstr=self.create_termrunsh(execfile,magics)
                if self.sys=='Windows':
                    cmd=magics['_st']['term']+[cmdshstr]
                elif self.sys=='Linux':
                    cmd=magics['_st']['term']+['--',cmdshstr]
                else:
                    cmd=magics['_st']['term']+['--',cmdshstr]
            cstr=''
            for x in cmd: cstr+=x+" "
            self._logln(cstr)
            if(magics!=None and (outencode==None or len(outencode)<0)):
                outencode=self.get_outencode(magics)
            if(outencode==None or len(outencode)<0):
                outencode='UTF-8'
            return RealTimeSubprocess(cmd,
                                  self._write_to_stdout,
                                  self._write_to_stderr,
                                  self._read_from_stdin,cwd,shell,env,self,outencode=outencode)
        except Exception as e:
            self._logln("RealTimeSubprocess err:"+str(e),3)
            raise
    def getossubsys(self):
        uname=''
        try:
            u=os.popen('bash -c "uname"')
            uname=u.read()
        except Exception as e:
            self._logln(""+str(e),3)
        return uname
    def inittermcmd(self,magics):
        if len(magics['_st']['term'])>0:return ''
        termcmd=''
        try:
            if self.subsys.startswith('MINGW64') or self.subsys.startswith('CYGWIN'):
                termcmd='mintty "/usr/bin/bash" --login'
            if self.sys=='Linux':
                termcmd='gnome-terminal'
            elif self.sys=='Windows':
                termcmd='c:\\Windows\\System32\\cmd.exe /c start'
        except Exception as e:
            self._logln(""+str(e),3)
        if len(termcmd)>1:
            magics['_st']['term']=[]
            for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', termcmd):
                magics['_st']['term'] += [argument.strip('"')]
        return termcmd
## %overwritefile
## %file:../src/create_termrunsh.py
## %noruncode
    def create_termrunsh(self,execfile,magics):
        fil_ename=execfile
        uname=''
        try:
            u=os.popen('bash -c "uname"')
            uname=u.read()
        except Exception as e:
            self._logln(""+str(e),3)
        if self.subsys.startswith('MINGW64') or self.subsys.startswith('CYGWIN'):
            pausestr=self.pausestr
            termrunsh="\n"+execfile+"\n"+pausestr+"\n"
            termrunsh_file=self.create_codetemp_file(magics,termrunsh,suffix='.sh')
            newsrcfilename=termrunsh_file.name
            fil_ename=newsrcfilename
        elif self.sys=='Windows' :
            termrunsh="echo off\r\ncls\r\n"+execfile+"\r\npause\r\nexit\r\n"
            if execfile.strip().lower().endswith(".bat"):
                termrunsh="echo off\r\ncls\r\ncall "+execfile+"\r\npause\r\nexit\r\n"
            termrunsh_file=self.create_codetemp_file(magics,termrunsh,suffix='.bat')
            newsrcfilename=termrunsh_file.name
            fil_ename=newsrcfilename
        elif self.sys=='Linux':
            pausestr=self.pausestr
            termrunsh="\n"+execfile+"\n"+pausestr+"\n"
            termrunsh_file=self.create_codetemp_file(magics,termrunsh,suffix='.sh')
            newsrcfilename=termrunsh_file.name
            fil_ename=newsrcfilename
        else:
            self._logln("Cannot create terminal!",3)
        self._logln(fil_ename)
        os.chmod(newsrcfilename,stat.S_IRWXU+stat.S_IRGRP+stat.S_IXGRP+stat.S_IXOTH)
        return fil_ename


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/create_jupyter_subprocess.py created successfully


In [25]:
## compile_with_Python2native
##%overwritefile
##%file:../src/compile_with_Python2native.py
##%noruncode
    def compile_with_Python2native(self, source_filename, binary_filename, cflags=None, ldflags=None):
        # outfile=None
        # outfile= ['-o', binary_filename]
        args = ['pyinstaller','-F', source_filename] + cflags + ldflags
        # for x in args: self._write_to_stderr(" " + x + " ")
        return self.create_jupyter_subprocess(args),source_filename


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/compile_with_Python2native.py created successfully


In [26]:
## _add_main
##%overwritefile
##%file:../src/_add_main.py
##%noruncode
    def _add_main(self, magics, code):
        # remove comments
        tmpCode = re.sub(r"//.*", "", code)
        tmpCode = re.sub(r"/\*.*?\*/", "", tmpCode, flags=re.M|re.S)

        x = re.search(r".*\s+main\s*\(", tmpCode)

        if not x:
            code = self.main_head + code + self.main_foot
            # magics['_st']['cflags'] += ['-lm']

        return magics, code


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_add_main.py created successfully


In [27]:
## do_shutdown
##%overwritefile
##%file:../src/do_shutdown.py
##%noruncode
    def do_shutdown(self, restart):
        self.g_chkreplexit=False
        self.chk_replexit_thread.join()
        # self.onkernelshutdown()
        """Cleanup the created source code files and executables when shutting down the kernel"""
        self.cleanup_files()


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/do_shutdown.py created successfully
